In [50]:
from tqdm.notebook import tqdm
from pprint import pprint
from glob import glob
import random
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

In [1]:
! unzip wiki_data

Archive:  wiki_data.zip
   creating: wiki_data/
  inflating: wiki_data/texts390.txt  
  inflating: wiki_data/texts975.txt  
  inflating: wiki_data/texts60.txt   
  inflating: wiki_data/texts774.txt  
  inflating: wiki_data/texts373.txt  
  inflating: wiki_data/texts728.txt  
  inflating: wiki_data/texts852.txt  
  inflating: wiki_data/texts418.txt  
  inflating: wiki_data/texts634.txt  
  inflating: wiki_data/texts311.txt  
  inflating: wiki_data/texts1024.txt  
  inflating: wiki_data/texts858.txt  
  inflating: wiki_data/texts4.txt    
  inflating: wiki_data/texts86.txt   
  inflating: wiki_data/texts775.txt  
  inflating: wiki_data/texts869.txt  
  inflating: wiki_data/texts613.txt  
  inflating: wiki_data/texts849.txt  
  inflating: wiki_data/texts442.txt  
  inflating: wiki_data/texts871.txt  
  inflating: wiki_data/texts415.txt  
  inflating: wiki_data/texts696.txt  
  inflating: wiki_data/texts937.txt  
  inflating: wiki_data/texts17.txt   
  inflating: wiki_data/texts646.txt  
 

  inflating: wiki_data/texts274.txt  
  inflating: wiki_data/texts386.txt  
  inflating: wiki_data/texts87.txt   
  inflating: wiki_data/texts737.txt  
  inflating: wiki_data/texts277.txt  
  inflating: wiki_data/texts791.txt  
  inflating: wiki_data/texts1018.txt  
  inflating: wiki_data/texts478.txt  
  inflating: wiki_data/texts897.txt  
  inflating: wiki_data/texts343.txt  
  inflating: wiki_data/texts100.txt  
  inflating: wiki_data/texts565.txt  
  inflating: wiki_data/texts420.txt  
  inflating: wiki_data/texts81.txt   
  inflating: wiki_data/texts46.txt   
  inflating: wiki_data/texts12.txt   
  inflating: wiki_data/texts248.txt  
  inflating: wiki_data/texts52.txt   
  inflating: wiki_data/texts865.txt  
  inflating: wiki_data/texts354.txt  
  inflating: wiki_data/texts241.txt  
  inflating: wiki_data/texts70.txt   
  inflating: wiki_data/texts39.txt   
  inflating: wiki_data/texts997.txt  
  inflating: wiki_data/texts227.txt  
  inflating: wiki_data/texts755.txt  
  inflating

  inflating: wiki_data/texts36.txt   
  inflating: wiki_data/texts896.txt  
  inflating: wiki_data/texts607.txt  
  inflating: wiki_data/texts1021.txt  
  inflating: wiki_data/texts289.txt  
  inflating: wiki_data/texts345.txt  
  inflating: wiki_data/texts290.txt  
  inflating: wiki_data/texts258.txt  
  inflating: wiki_data/texts933.txt  
  inflating: wiki_data/texts808.txt  
  inflating: wiki_data/texts603.txt  
  inflating: wiki_data/texts816.txt  
  inflating: wiki_data/texts347.txt  
  inflating: wiki_data/texts1033.txt  
  inflating: wiki_data/texts312.txt  
  inflating: wiki_data/texts601.txt  
  inflating: wiki_data/texts578.txt  
  inflating: wiki_data/texts191.txt  
  inflating: wiki_data/texts702.txt  
  inflating: wiki_data/texts742.txt  
  inflating: wiki_data/texts543.txt  
  inflating: wiki_data/texts908.txt  
  inflating: wiki_data/texts801.txt  
  inflating: wiki_data/texts426.txt  
  inflating: wiki_data/texts159.txt  
  inflating: wiki_data/texts476.txt  
  inflatin

  inflating: wiki_data/texts514.txt  
  inflating: wiki_data/texts346.txt  
  inflating: wiki_data/texts459.txt  
  inflating: wiki_data/texts573.txt  
  inflating: wiki_data/texts584.txt  
  inflating: wiki_data/texts448.txt  
  inflating: wiki_data/texts375.txt  
  inflating: wiki_data/texts462.txt  
  inflating: wiki_data/texts221.txt  
  inflating: wiki_data/texts257.txt  
  inflating: wiki_data/texts927.txt  
  inflating: wiki_data/texts5.txt    
  inflating: wiki_data/texts214.txt  
  inflating: wiki_data/texts889.txt  
  inflating: wiki_data/texts928.txt  
  inflating: wiki_data/texts632.txt  
  inflating: wiki_data/texts895.txt  
  inflating: wiki_data/texts47.txt   
  inflating: wiki_data/texts763.txt  
  inflating: wiki_data/texts153.txt  
  inflating: wiki_data/texts553.txt  
  inflating: wiki_data/texts179.txt  
  inflating: wiki_data/texts206.txt  
  inflating: wiki_data/texts305.txt  
  inflating: wiki_data/texts422.txt  
  inflating: wiki_data/texts582.txt  
  inflating:

In [52]:
documents = []

for fname in tqdm(glob('wiki_data/texts*.txt')):
    with open(fname) as f:
        document = ""
        for line in f:
            document = document + line.strip()
        
        documents.append(document)

len(documents), documents[0]

  0%|          | 0/1047 [00:00<?, ?it/s]

(1047,
 '= Carmichael number =In number theory, a Carmichael number is a composite numbern{\\displaystyle n}which satisfies the modular arithmetic congruence relation:bn−1≡1(modn){\\displaystyle b^{n-1}\\equiv 1{\\pmod {n}}}for all integersb{\\displaystyle b}which are relatively prime ton{\\displaystyle n}.They are named for Robert Carmichael.The Carmichael numbers are the subset K1 of the Knödel numbers.Equivalently, a Carmichael number is a composite numbern{\\displaystyle n}for whichbn≡b(modn){\\displaystyle b^{n}\\equiv b{\\pmod {n}}}for all integersb{\\displaystyle b}.OverviewFermat\'s little theorem states that if p is a prime number, then for any integer b, the number bp − b is an integer multiple of p.  Carmichael numbers are composite numbers which have this property.  Carmichael numbers are also called Fermat pseudoprimes or absolute Fermat pseudoprimes. A Carmichael number will pass a Fermat primality test to every base b relatively prime to the number, even though it is not

Training the model

In [53]:
nlp = spacy.load("en", disable=["parser", "ner", "tagger"])

def spacy_tokenizer(text):
    return [t.lemma_ for t in nlp(text)]

vec = TfidfVectorizer(tokenizer=spacy_tokenizer)
trained_vectors = vec.fit_transform(documents).todense()
texts = [[document, vector] for document, vector in zip(documents, trained_vectors)]

Searching

In [60]:
def do_search(text):
    new_vector = vec.transform([text]).todense()
    texts_tmp = [[cosine(vector, new_vector), document] for document, vector in texts]
    texts_tmp.sort()
    return [document for vector, document in texts_tmp[:10]]

do_search('Dijkstra')

['= Dijkstra\'s algorithm =Dijkstra\'s algorithm ( DYKE-strəz) is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks.  It was conceived by computer scientist Edsger W. Dijkstra in 1956 and published three years later.The algorithm exists in many variants. Dijkstra\'s original algorithm found the shortest path between two given nodes, but a more common variant fixes a single node as the "source" node and finds shortest paths from the source to all other nodes in the graph, producing a shortest-path tree.For a given source node in the graph, the algorithm finds the shortest path between that node and every other.:\u200a196–206\u200a It can also be used for finding the shortest paths from a single node to a single destination node by stopping the algorithm once the shortest path to the destination node has been determined. For example, if the nodes of the graph represent cities and edge path costs represent driving distanc

In [64]:
s = input()
do_search(s)

Trie


['= Burstsort =Burstsort and its variants are cache-efficient algorithms for sorting strings.  They are variants of the traditional radix sort but faster for large data sets of common strings, first published in 2003, with some optimizing versions published in later years.Burstsort algorithms use a trie to store prefixes of strings, with growable arrays of pointers as end nodes containing sorted, unique, suffixes (referred to as buckets). Some variants copy the string tails into the buckets. As the buckets grow beyond a predetermined threshold, the buckets are "burst" into tries, giving the sort its name. A more recent variant uses a bucket index with smaller sub-buckets to reduce memory usage. Most implementations delegate to multikey quicksort, an extension of three-way radix quicksort, to sort the contents of the buckets. By dividing the input into buckets with common prefixes, the sorting can be done in a cache-efficient manner.Burstsort was introduced as a sort that is similar to 